In [132]:
%load_ext autoreload
%autoreload 2

In [149]:
from __future__ import print_function, absolute_import

import numpy as np
import matplotlib.pyplot as plt

import pydrake
from pydrake.all import (
    SolutionResult
)
from pydrake.solvers import mathematicalprogram as mp
from pydrake.solvers.mathematicalprogram import SolverType

from NNSystem import NNSystem, NNSystem_
from NNSystemHelper import FC, MLP, NNInferenceHelper

prog = mp.MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")
# prog.AddLinearConstraint(x[0] >= 1)
# prog.AddLinearConstraint(x[1] >= 1)
# prog.AddQuadraticCost(np.eye(2), np.zeros(2), x)
# Redundant cost just to check the spelling.
# prog.AddQuadraticErrorCost(vars=x, Q=np.eye(2),
#                            x_desired=np.zeros(2))
# prog.AddL2NormCost(A=np.eye(2), b=np.zeros(2), vars=x)
    
# net = FC()
net = MLP()

# This function uses an neural network, but only has one output...
def autodiffable_function_nn(drake_in):
    global net
    in_list = list(drake_in)
    out_list = NNInferenceHelper(net, in_list)
    return out_list[0]

def autodiffable_function_simple(x):
    return (x[0]-2.)*(x[1]-2.)

# prog.AddCost(autodiffable_function_simple, x)
prog.AddCost(autodiffable_function_nn, x)

result = prog.Solve()
print(result, mp.SolutionResult.kSolutionFound)
x_expected = np.array([1, 1])
print(np.allclose(prog.GetSolution(x), x_expected))
print(prog.GetSolution(x))

('torch_in: ', tensor([0., 0.], requires_grad=True))
('torch_out: ', tensor([-0.0015], grad_fn=<AddBackward0>))
('\niter j: ', 0)
('dy_jdu_a[i] * u_i_deriv = ', -0.010617537468229517, ' * ', array([1., 0.]))
('dy_jdu_a[i] * u_i_deriv = ', 0.05061211794189872, ' * ', array([0., 1.]))
('putting into output: ', array(-0.00150494), ', ', array([-0.01061754,  0.05061212]))
('torch_in: ', tensor([ 2.7387e-07, -2.7387e-07], requires_grad=True))
('torch_out: ', tensor([-0.0015], grad_fn=<AddBackward0>))
('\niter j: ', 0)
('dy_jdu_a[i] * u_i_deriv = ', -0.010617544771119092, ' * ', array([1., 0.]))
('dy_jdu_a[i] * u_i_deriv = ', 0.05061211671054929, ' * ', array([0., 1.]))
('putting into output: ', array(-0.00150496), ', ', array([-0.01061754,  0.05061212]))
('torch_in: ', tensor([ 0.0102, -0.0485], requires_grad=True))
('torch_out: ', tensor([-0.0040], grad_fn=<AddBackward0>))
('\niter j: ', 0)
('dy_jdu_a[i] * u_i_deriv = ', -0.012202912129759928, ' * ', array([1., 0.]))
('dy_jdu_a[i] * u_i_de